# Project

In [2]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import uniform, randint, ttest_rel, ttest_ind

from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = 10

import warnings
warnings.filterwarnings(action='ignore')

import matplotlib.pyplot as plt
%matplotlib inline

import chart_studio.plotly as py
import plotly.graph_objects as go
import plotly.express as px

# from pandas_profiling import ProfileReport

from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif 
from sklearn.feature_selection import SelectPercentile, VarianceThreshold, SelectFromModel, RFE

from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score, r2_score, mean_squared_error
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold, StratifiedKFold, RepeatedKFold, RepeatedStratifiedKFold, LeaveOneOut
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

#preprocessing:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, normalize, scale, Normalizer, MinMaxScaler, FunctionTransformer

# models:
from sklearn.decomposition import PCA, KernelPCA
from sklearn.cluster import FeatureAgglomeration
from sklearn.ensemble import StackingClassifier
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [45]:
cognitive_abilities = ["BarChartLit", "Meara", "VerbalWM_longest"]
best_window = {"BarChartLit": 3, "Meara": 4, "VerbalWM_longest": 2}

cognitive_ability = cognitive_abilities[1]
task_number = best_window[cognitive_ability]

# loading the dataset
path = ("Data/"+ str(cognitive_ability) + "/Task_" + str(task_number) + ".csv")
df = pd.read_csv(path)

display(df)
print(df.shape)

,abspathanglesrate,enddistance,endpupilsize,eyemovementvelocity,fixationrate,fixationsaccadetimeratio,longestsaccadedistance,longestsaccadeduration,maxdistance,maxpupilsize,maxpupilvelocity,maxsaccadespeed,meanabspathangles,meandistance,meanfixationduration,meanpathdistance,meanpupilsize,meanpupilvelocity,meanrelpathangles,meansaccadedistance,meansaccadeduration,meansaccadespeed,mindistance,minpupilsize,minpupilvelocity,minsaccadespeed,relpathanglesrate,startdistance,startpupilsize,stddevabspathangles,stddevdistance,stddevfixationduration,stddevpathdistance,stddevpupilsize,stddevpupilvelocity,stddevrelpathangles,stddevsaccadedistance,stddevsaccadeduration,stddevsaccadespeed,Relevant.bars_fixationrate,Relevant.bars_leftclicrate,Relevant.bars_longestfixation,Relevant.bars_maxdistance,Relevant.bars_maxpupilsize,Relevant.bars_maxpupilvelocity,Relevant.bars_meandistance,Relevant.bars_meanfixationduration,Relevant.bars_meanpupilsize,Relevant.bars_meanpupilvelocity,Relevant.bars_mindistance,Relevant.bars_minpupilsize,Relevant.bars_minpupilvelocity,Relevant.bars_numtransfrom_Non.relevant.bars,Relevant.bars_numtransfrom_Refs,Relevant.bars_numtransfrom_Relevant.bars,Relevant.bars_numtransfrom_Text,Relevant.bars_numtransfrom_Viz,Relevant.bars_numtransfrom_labels,Relevant.bars_numtransfrom_legend,Relevant.bars_proportionnum,Relevant.bars_proportiontime,Relevant.bars_proptransfrom_Non.relevant.bars,Relevant.bars_proptransfrom_Refs,Relevant.bars_proptransfrom_Relevant.bars,Relevant.bars_proptransfrom_Text,Relevant.bars_proptransfrom_Viz,Relevant.bars_proptransfrom_labels,Relevant.bars_proptransfrom_legend,Relevant.bars_stddevdistance,Relevant.bars_stddevfixationduration,Relevant.bars_stddevpupilsize,Relevant.bars_stddevpupilvelocity,Relevant.bars_timetofirstfixation,Non.relevant.bars_fixationrate,Non.relevant.bars_leftclicrate,Non.relevant.bars_longestfixation,Non.relevant.bars_maxdistance,Non.relevant.bars_maxpupilsize,Non.relevant.bars_maxpupilvelocity,Non.relevant.bars_meandistance,Non.relevant.bars_meanfixationduration,Non.relevant.bars_meanpupilsize,Non.relevant.bars_meanpupilvelocity,Non.relevant.bars_mindistance,Non.relevant.bars_minpupilsize,Non.relevant.bars_minpupilvelocity,Non.relevant.bars_numtransfrom_Non.relevant.bars,Non.relevant.bars_numtransfrom_Refs,Non.relevant.bars_numtransfrom_Relevant.bars,Non.relevant.bars_numtransfrom_Text,Non.relevant.bars_numtransfrom_Viz,Non.relevant.bars_numtransfrom_labels,Non.relevant.bars_numtransfrom_legend,Non.relevant.bars_proportionnum,Non.relevant.bars_proportiontime,Non.relevant.bars_proptransfrom_Non.relevant.bars,Non.relevant.bars_proptransfrom_Refs,Non.relevant.bars_proptransfrom_Relevant.bars,Non.relevant.bars_proptransfrom_Text,Non.relevant.bars_proptransfrom_Viz,Non.relevant.bars_proptransfrom_labels,Non.relevant.bars_proptransfrom_legend,Non.relevant.bars_stddevdistance,Non.relevant.bars_stddevfixationduration,Non.relevant.bars_stddevpupilsize,Non.relevant.bars_stddevpupilvelocity,Non.relevant.bars_timetofirstfixation,Text_fixationrate,Text_leftclicrate,Text_longestfixation,Text_maxdistance,Text_maxpupilsize,Text_maxpupilvelocity,Text_meandistance,Text_meanfixationduration,Text_meanpupilsize,Text_meanpupilvelocity,Text_mindistance,Text_minpupilsize,Text_minpupilvelocity,Text_numtransfrom_Non.relevant.bars,Text_numtransfrom_Refs,Text_numtransfrom_Relevant.bars,Text_numtransfrom_Text,Text_numtransfrom_Viz,Text_numtransfrom_labels,Text_numtransfrom_legend,Text_proportionnum,Text_proportiontime,Text_proptransfrom_Non.relevant.bars,Text_proptransfrom_Refs,Text_proptransfrom_Relevant.bars,Text_proptransfrom_Text,Text_proptransfrom_Viz,Text_proptransfrom_labels,Text_proptransfrom_legend,Text_stddevdistance,Text_stddevfixationduration,Text_stddevpupilsize,Text_stddevpupilvelocity,Text_timetofirstfixation,Refs_fixationrate,Refs_leftclicrate,Refs_longestfixation,Refs_maxdistance,Refs_maxpupilsize,Refs_maxpupilvelocity,Refs_meandistance,Refs_meanfixationduration,Refs_meanpupilsize,Refs_meanp

(53, 278)


In [46]:
target = cognitive_ability

X = df.drop(columns=[target])
y = df[target]

# baseline score
DC = DummyClassifier(strategy="most_frequent")
DC.fit(X, y)
DC.score(X, y)

0.5094339622641509

# Functions

In [5]:
#--------------------------------------------------------------------------------------

def results(y, y_pred):
    # confusion_matatrix
    # TN, FP, FN, TP = confusion_matatrix.ravel()
    confusion_matatrix = confusion_matrix(y, y_pred)
    report = classification_report(y, y_pred, digits=5, target_names=['Low', 'High'])

    class_accuracy = confusion_matatrix.diagonal()/confusion_matatrix.sum(axis=1)
    overall_accuracy = confusion_matatrix.diagonal().sum()/confusion_matatrix.sum()

    data = {'Overall': overall_accuracy, 'Low': class_accuracy[0], 'High': class_accuracy[1]}
    results = pd.DataFrame(data, index = ['Accuracy'])

    display(results)
    # print(report)
    # print(confusion_matatrix)
    
#--------------------------------------------------------------------------------------

def results2(name, y, y_pred):

    confusion_matatrix = confusion_matrix(y, y_pred)
    class_accuracy = confusion_matatrix.diagonal()/confusion_matatrix.sum(axis=1)
    overall_accuracy = confusion_matatrix.diagonal().sum()/confusion_matatrix.sum()

    data = {'Overall': overall_accuracy, 'Low': class_accuracy[0], 'High': class_accuracy[1]}
    results = pd.DataFrame(data, index = [name])

    return results
    
#--------------------------------------------------------------------------------------

def ttest_results(y, y_pred_before, y_pred_after):
    
    low_before     = (np.array(y[y==0]) == y_pred_before[y==0]).astype(int)
    high_before    = (np.array(y[y==1]) == y_pred_before[y==1]).astype(int)
    overall_before = (np.array(y) == y_pred_before).astype(int)

    low_after     = (np.array(y[y==0]) == y_pred_after[y==0]).astype(int)
    high_after    = (np.array(y[y==1]) == y_pred_after[y==1]).astype(int)
    overall_after = (np.array(y) == y_pred_after).astype(int)

    low_class  = ttest_rel(low_before, low_after)
    high_class = ttest_rel(high_before, high_after)
    overall    = ttest_rel(overall_before, overall_after)

    print("low    :", low_class)
    print("high   :", high_class)
    print("overall:", overall)
    
#--------------------------------------------------------------------------------------

# Classifiers/ hyper-parameter distributions

In [6]:
#--------------------------------------------------------------------------------------
class_weight = {0:1, 1:1}
seed = None

#--------------------------------------------------------------------------------------
# models:

classifiers = {
    #'rf'  : RandomForestClassifier(random_state=seed, max_depth=2),
    'lr'  : LogisticRegression(solver='liblinear'),
    'svm' : SVC(gamma='auto'),
    #'xgb' : GradientBoostingClassifier(),
    'knn' : KNeighborsClassifier(),
    #'gpc' : GaussianProcessClassifier(),
    #'ada' : AdaBoostClassifier(LogisticRegression(solver='liblinear'), n_estimators=100),
    #'gnb' : GaussianNB(),
    #'qda' : QuadraticDiscriminantAnalysis()
}

estimators = [
    #('rf', RandomForestClassifier()),
    ('lr', LogisticRegression(solver='liblinear', class_weight = {0:10, 1:1})),
    ('lr2', LogisticRegression(solver='liblinear', class_weight = {0:1, 1:10})),
    ('lr3', LogisticRegression(solver='liblinear', C=0.1)),
    ('lr4', LogisticRegression(solver='liblinear', C=1)),
    ('lr5', LogisticRegression(solver='liblinear', C=10)),
    ('knn', KNeighborsClassifier()),
    #('ada', classifiers['ada']),
    ('svm', SVC(gamma='auto', class_weight = {0:10, 1:1})),
    ('svm2', SVC(gamma='auto', C=1)),
    ('svm3', SVC(gamma='auto', C=10)),
    ('svm4', SVC(gamma='auto', C=0.1)),
]
              
ensemble = StackingClassifier(estimators=estimators, passthrough=True, n_jobs=-1,
                              final_estimator=LogisticRegression())

#--------------------------------------------------------------------------------------
# hyper-parameters

lr_dist = {
    'estimator__C'       : [0.1, 1, 10],
    'estimator__penalty' : ['l1', 'l2']}
    
rf_dist = {
    'estimator__n_estimators'   : randint(low=10, high=300),
    'estimator__max_depth'      : randint(low=10, high=30)}

knn_dist = {
    'estimator__n_neighbors' : [2, 3, 4, 5, 6, 7, 8, 9, 10]}

svm_dist = {
    'estimator__C' : [0.1, 1, 10]}

distributions = {
    'rf'  : rf_dist,
    'lr'  : lr_dist,
    'knn' : knn_dist,
    'svm' : svm_dist,
    'gpc' : None,
    'ada' : None
}

#--------------------------------------------------------------------------------------
model = 'lr'
estimator = classifiers[model]
# parameters = distributions[model]

estimator = ensemble

# Pipelines

In [7]:
#--------------------------------------------------------------------------------------
# pipeline

preprocessing = Pipeline([
    ('vart', VarianceThreshold(threshold=0.0)),
    #('normalize', Normalizer()),
    ('scaler', StandardScaler()),
    ('minmax', MinMaxScaler())
])

feature_selection = Pipeline([
    ('sp', SelectPercentile(percentile=100)),
    ('kbest', SelectKBest(chi2, k=10)),
    #('rfe', RFE(RandomForestClassifier(), n_features_to_select=10, step=10))
    #('kbest', SelectKBest(mutual_info_classif, k=10)),
])

dim_reduction = Pipeline([
    ('pca', PCA(n_components=1)),
    #('fa', FeatureAgglomeration(n_clusters=2))
    #('kpca', KernelPCA(n_components=1, kernel='rbf'))
])

original = Pipeline([
    ('estimator', estimator)])

engineered = Pipeline([
    ('preprocessing', preprocessing),                              
    ('feature_selection', feature_selection),
    #('dim_reduction', dim_reduction),
    ('estimator', estimator)
])

In [319]:
loo = LeaveOneOut()
original_models = pd.DataFrame()
for name, estimator in classifiers.items():
    
    original = Pipeline([
    ('preprocessing', preprocessing),
    ('estimator', estimator)])
    
    y_pred  = cross_val_predict(original, X, y, cv=loo, n_jobs=-1, verbose=0)
    original_models = original_models.append(results2(name, y, y_pred))

display(original_models)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  53 out of  53 | elapsed:    2.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 out of  53 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  53 out of  53 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 out of  53 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  53 out of  53 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done  53 out of  53 | elapsed:    2.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 out of  53 | elaps

,Overall,Low,High
rf,0.641509,0.653846,0.629630
lr,0.584906,0.538462,0.629630
svm,0.377358,0.000000,0.740741
xgb,0.509434,0.538462,0.481481
knn,0.471698,0.538462,0.407407


In [321]:
engineered_models = pd.DataFrame()
for name, estimator in classifiers.items():
    
    engineered = Pipeline([
        ('preprocessing', preprocessing),                              
        ('feature_selection', feature_selection),
        #('dim_reduction', dim_reduction),
        ('estimator', estimator)])
    
    y_pred  = cross_val_predict(engineered, X, y, cv=loo, n_jobs=-1, verbose=0)
    engineered_models = engineered_models.append(results2(name, y, y_pred))
    
display(engineered_models)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done  53 out of  53 | elapsed:    3.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 out of  53 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  53 out of  53 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 out of  53 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  53 out of  53 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 out of  53 | elapsed:    0.6s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  53 out of  53 | elapsed:    0.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38

,Overall,Low,High
rf,0.622642,0.653846,0.592593
lr,0.679245,0.692308,0.666667
svm,0.641509,0.730769,0.555556
xgb,0.528302,0.538462,0.518519
knn,0.641509,0.576923,0.703704


In [328]:
#--------------------------------------------------------------------------------------
# outer cross-validation on model, inner cross-validation on hyperparameters
#--------------------------------------------------------------------------------------
tuned_models = pd.DataFrame()
for name, estimator in classifiers.items():
    
    engineered = Pipeline([
        ('preprocessing', preprocessing),                              
        ('feature_selection', feature_selection),
        ('estimator', estimator)])
    
    parameters = distributions[name]
    
    inner_loop = RandomizedSearchCV(engineered, parameters, n_iter=10, cv=loo, refit=True, n_jobs=-1, verbose=False)
    y_pred  = cross_val_predict(inner_loop, X, y, cv=loo, n_jobs=-1, verbose=1)
    tuned_models = tuned_models.append(results2(name, y, y_pred))
    
display(tuned_models)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   36.8s
[Parallel(n_jobs=-1)]: Done  53 out of  53 | elapsed:   51.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   17.9s
[Parallel(n_jobs=-1)]: Done  53 out of  53 | elapsed:   24.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   55.4s
[Parallel(n_jobs=-1)]: Done  53 out of  53 | elapsed:  1.3min finished


,Overall,Low,High
lr,0.622642,0.538462,0.703704
svm,0.641509,0.653846,0.629630
knn,0.603774,0.615385,0.592593


In [422]:
estimators = [
    ('lr', LogisticRegression(solver='liblinear', C=0.1, penalty='l1')),
    ('knn', KNeighborsClassifier()),
    ('knn3', KNeighborsClassifier(n_neighbors=3)),
    ('knn4', KNeighborsClassifier(n_neighbors=4)),
    ('svm', SVC(gamma='auto', class_weight = {0:10, 1:1})),
    ('svm2', SVC(gamma='auto', C=1)),
    ('svm3', SVC(gamma='auto', C=10)),
]
              
ensemble = StackingClassifier(estimators=estimators, passthrough=True, n_jobs=-1,
                              final_estimator=LogisticRegression())

estimator = ensemble

engineered = Pipeline([
        ('preprocessing', preprocessing),                              
        ('kbest', SelectKBest(chi2, k=10)),
        ('estimator', estimator)])

y_pred  = cross_val_predict(engineered, X, y, cv=loo, n_jobs=-1, verbose=1)
ensembled = (results2('ensemble', y, y_pred))
display(ensembled)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  53 out of  53 | elapsed:    1.7s finished


,Overall,Low,High
ensemble,0.735849,0.730769,0.740741


In [157]:
loo = LeaveOneOut()

estimators = [
    #('rf', RandomForestClassifier(random_state=seed, max_depth=None)),
    #('xgb', GradientBoostingClassifier())
    ('lr', LogisticRegression()),
    ('knn', KNeighborsClassifier(n_neighbors=10)),
    ('knn3', KNeighborsClassifier(n_neighbors=5)),
    #('knn4', KNeighborsClassifier(n_neighbors=4)),
    #('svm', SVC()),
    #('svm2', SVC(gamma='auto', C=1)),
    #('svm3', SVC(gamma='auto', C=10)),
    #('gpc', GaussianProcessClassifier()),
    #('ada', AdaBoostClassifier(LogisticRegression(solver='liblinear'), n_estimators=100)),
    #('gnb', GaussianNB()),
    #('qda', QuadraticDiscriminantAnalysis())
]
              
ensemble = StackingClassifier(estimators=estimators, passthrough=True, n_jobs=-1,
                              final_estimator=LogisticRegression())

estimator = ensemble

engineered = Pipeline([
        ('preprocessing', preprocessing),                              
        ('kbest', SelectKBest(chi2, k=10)),
        #('pca', PCA(n_components=10)),
        ('estimator', estimator)])

y_pred  = cross_val_predict(engineered, X, y, cv=loo, n_jobs=-1, verbose=1)
ensembled = (results2('ensemble', y, y_pred))
display(ensembled)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  54 out of  54 | elapsed:    1.6s finished


,Overall,Low,High
ensemble,0.703704,0.692308,0.714286


In [54]:
loo = LeaveOneOut()

estimators = [
    #('rf', RandomForestClassifier(random_state=seed, max_depth=None)),
    #('xgb', GradientBoostingClassifier()),
    ('lr', LogisticRegression(C=10)),
    ('lr2', LogisticRegression(C=1)),
    ('lr3', LogisticRegression(C=0.1)),
    ('knn', KNeighborsClassifier(n_neighbors=10)),
    ('knn3', KNeighborsClassifier(n_neighbors=5)),
    ('knn4', KNeighborsClassifier(n_neighbors=4)),
    ('svm', SVC(C=0.1)),
    ('svm2', SVC(gamma='auto', C=1)),
    ('svm3', SVC(gamma='auto', C=10)),
    #('gpc', GaussianProcessClassifier()),
    #('ada', AdaBoostClassifier(LogisticRegression(solver='liblinear'), n_estimators=100)),
    #('gnb', GaussianNB()),
    #('qda', QuadraticDiscriminantAnalysis())
]
              
ensemble = StackingClassifier(estimators=estimators, passthrough=True, n_jobs=-1,
                              final_estimator=LogisticRegression())

estimator = LogisticRegression()

parameters = {
    'C': Real(1e-6, 1e+6, prior='log-uniform'),
}

engineered = Pipeline([
        ('preprocessing', preprocessing),                              
        ('kbest', SelectKBest(chi2, k=10)),
        #('pca', PCA(n_components=10)),
        ('estimator', estimator)])

ensembled = pd.DataFrame()

for i in range(0):
    inner_loop = BayesSearchCV(estimator, parameters, n_iter=10, n_points=1, cv=10, refit=True, n_jobs=-1, verbose=False)
    y_pred  = cross_val_predict(inner_loop, X, y, cv=loo, n_jobs=-1, verbose=1)
    ensembled = ensembled.append(results2('ensemble', y, y_pred))
    

inner_loop = BayesSearchCV(estimator, parameters, n_iter=10, n_points=1, cv=10, refit=True, n_jobs=-1, verbose=False)
inner_loop.fit(X, y)
#outer_loop = cross_validate(inner_loop, X, y, cv=loo, return_train_score=True, 
#                            n_jobs=-1, verbose=1, return_estimator=True)

ensembled = ensembled.append(results2('ensemble', y, y_pred))

display(ensembled)

,Overall,Low,High
ensemble,0.54717,0.538462,0.555556


In [56]:
# display(ensembled.mean(axis=0))
#for i in range(53):
#   print(outer_loop['estimator'][i].best_estimator_.get_params()['C'])

inner_loop.cv_results_

defaultdict(list,
            {'split0_test_score': [0.6666666666666666,
              0.5,
              0.6666666666666666,
              0.6666666666666666,
              0.5,
              0.5,
              0.5,
              0.5,
              0.6666666666666666,
              0.6666666666666666],
             'split1_test_score': [0.5,
              0.5,
              0.5,
              0.5,
              0.3333333333333333,
              0.5,
              0.5,
              0.3333333333333333,
              0.3333333333333333,
              0.5],
             'split2_test_score': [0.5,
              0.5,
              0.5,
              0.5,
              0.5,
              0.5,
              0.5,
              0.5,
              0.5,
              0.5],
             'split3_test_score': [0.4,
              0.4,
              0.4,
              0.4,
              0.4,
              0.4,
              0.2,
              0.2,
              0.4,
              0.2],
             '

In [170]:
# loo = LeaveOneOut()

# inner_loop = GridSearchCV(pipeline, parameters, cv=inner_rkf, refit=True, n_jobs=-1, verbose=False)
# inner_loop = RandomizedSearchCV(pipeline, parameters, n_iter=10, cv=inner_rkf, refit=True, n_jobs=-1, verbose=False)
# inner_loop = BayesSearchCV(pipeline, parameters, n_iter=2, n_points=2, cv=10, refit=True, n_jobs=-1, verbose=False)

# outer_loop = cross_validate(pipeline, X, y, cv=loo, return_train_score=True, 
#                             n_jobs=-1, verbose=1, return_estimator=True)
# outer_loop['estimator'][0].best_estimator_

# y_pred_before = cross_val_predict(original, X, y, cv=loo, n_jobs=-1, verbose=1)
#y_pred_after  = cross_val_predict(engineered, X, y, cv=loo, n_jobs=-1, verbose=1)

#--------------------------------------------------------------------------------------

In [421]:
# print("train score: ", np.mean(outer_loop['train_score']))
# print("test score:  ", np.mean(outer_loop['test_score']))

# results(y, y_pred_before)
# results(y, y_pred_after)

# ttest_results(y, y_pred_before, y_pred_after)

# To Do:
2. model names
3. parmeter set for each model
4. statistical analysis

In [102]:
from sklearn.manifold import TSNE
# X_embedded = TSNE(n_components=2).fit_transform(X)

engineered = Pipeline([
    ('preprocessing', preprocessing),                              
    ('feature_selection', SelectKBest(chi2, k=10)),
    #('estimator', estimator)
])



X_embedded = engineered.fit_transform(X, y)
X_embedded = PCA(n_components=2).fit_transform(X)

# fig = go.Figure(data=[go.Scatter3d(x=X_embedded[:, 0], y=X_embedded[:, 1], z=X_embedded[:, 2],
#                                    mode='markers', marker=dict(size=5, color=y, opacity=0.8))])
fig = go.Figure(data=[go.Scatter(x=X_embedded[:, 0], y=X_embedded[:, 1],
                                  mode='markers', marker=dict(size=5, color=y, opacity=0.8))])
# fig.show()

engineered = Pipeline([
    ('preprocessing', preprocessing),                              
    ('feature_selection', feature_selection),
    #('dim_reduction', dim_reduction)
])



In [60]:
d = pd.DataFrame()
d.loc['1', '3']=3
d

d.to_csv(r'C:\Users\Alire\Desktop\DF.csv', index=True)